# Training a road sign detector with tensorflow 1.x (GPU ENABLED)

I have created this notebook with the help of this article
> https://medium.com/analytics-vidhya/training-an-object-detection-model-with-tensorflow-api-using-google-colab-4f9a688d5e8b


The dataset that I will be using can be found from here 
> http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset


And the function that helped me generate the tfrecords was found from here 
> https://github.com/datitran/raccoon_dataset/blob/master/generate_tfrecord.py

In [0]:
%tensorflow_version 1.x

# Enable GPU acceleration by navigating to 

> Edit > Notebook Settings > Choose "GPU" for hardware acceleration.

Then run the cell below to make sure the GPU is setup. You should see 'Found GPU'

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()

if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')

print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# **Mount** your google drive my using **'Mount drive'** from the files section.

Change directory to the folder you created on your google drive. I used 'RoadSignDetection'. If you used a different name replace every instance of 'RoadSignDetection' to the name you have chosen.

In [0]:
%cd 'drive/My Drive/RoadSignDetection/'

/content/drive/My Drive/RoadSignDetection


In [0]:
!pwd

/content/drive/My Drive/RoadSignDetection


Clone the Tensorflow models repository. Also you may clone the COCO repository and install the COCO object detection API for evaluation purpose.

In [0]:
!git clone https://github.com/tensorflow/models.git
%cd 'models/research'
!pip install "git+https://github.com/philferriere/cocoapi.git#egg=pycocotools&subdirectory=PythonAPI"
%cd '/content/drive/My Drive/RoadSignDetection/'

Cloning into 'models'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 34372 (delta 4), reused 27 (delta 0), pack-reused 34340
Receiving objects: 100% (34372/34372), 512.11 MiB | 22.91 MiB/s, done.
Resolving deltas: 100% (22059/22059), done.
Checking out files: 100% (2486/2486), done.
/content/drive/My Drive/RoadSignDetection
/content/drive/My Drive/RoadSignDetection/models/research
/content/drive/My Drive/RoadSignDetection


Install some needed tools.

In [0]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
python-tk is already the newest version (2.7.17-1~18.04).
The following additional packages will be installed:
  python-bs4 python-chardet python-html5lib python-olefile
  python-pkg-resources python-six python-webencodings
Suggested packages:
  python-genshi python-lxml-dbg python-lxml-doc python-pil-doc python-pil-dbg
  python-setuptools
The following NEW packages will be installed:
  python-bs4 python-chardet python-html5lib python-lxml python-olefile
  python-pil python-pkg-resources python-six python-webencodings
0 upgraded, 9 newly installed, 0 to remove and 25 not upgraded.
Need to get 1,818 kB of archives.
After this operation, 7,685 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python-bs4 all 4.6.0-1 [67.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd

Compile the model definition. Remember to change to the name of your folder.

In [0]:
%cd /content/drive/My Drive/RoadSignDetection/models/research/
!protoc object_detection/protos/*.proto --python_out=.

/content/drive/My Drive/RoadSignDetection/models/research


Set the environment

In [0]:
import os
os.environ['PYTHONPATH'] += ':/content/drive/My Drive/RoadSignDetection/models/research/:/content/drive/My Drive/RoadSignDetection/models/research/slim'

# **Always run this every restart of session**

In [0]:
!python setup.py build
!python setup.py install

running build
running build_py
copying object_detection/protos/argmax_matcher_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/bipartite_matcher_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/anchor_generator_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/mean_stddev_box_coder_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/box_coder_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/model_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/box_predictor_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/multiscale_anchor_generator_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/calibration_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/optimizer_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/eval_pb2.py -> build/lib/

# Note: if you wish to know the remaining hours you have for your colab session run the copy and run the code below.

In [0]:
import time, psutil
Start = time.time()- psutil.boot_time()
Left= 12*3600 - Start
print('Time remaining for this session is: ', round(Left/3600, 2), ' hours')

Time remaining for this session is:  11.83  hours


Test to see if all we need for the training has been installed.



In [0]:
!python object_detection/builders/model_builder_test.py

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Running tests under Python 3.6.9: /usr/bin/python3
[ RUN      ] ModelBuilderTest.test_create_experimental_model
[       OK ] ModelBuilderTest.test_create_experimental_model
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[       OK ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_with_matmul
[       OK ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_with_matmul
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_wi

# Up to here the training environment should be setup.

# Following we will insert the dataset that will be used for training, the tensorflow detection model and its .config file

In [0]:
%cd object_detection/

/content/drive/My Drive/RoadSignDetection/models/research/object_detection


Get the pre-trained Object detection model from tensorflow with the code below

More can be found from here:
> https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md


In [0]:
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz
!tar -xvf ssd_mobilenet_v2_coco_2018_03_29.tar.gz

--2020-03-05 19:08:11--  http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.11.80, 2607:f8b0:4007:80b::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.11.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 187925923 (179M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_coco_2018_03_29.tar.gz’

ssd_mobilenet_v2_co 100%[===================>] 179.22M  94.4MB/s    in 1.9s    

2020-03-05 19:08:13 (94.4 MB/s) - ‘ssd_mobilenet_v2_coco_2018_03_29.tar.gz’ saved [187925923/187925923]

ssd_mobilenet_v2_coco_2018_03_29/checkpoint
ssd_mobilenet_v2_coco_2018_03_29/model.ckpt.meta
ssd_mobilenet_v2_coco_2018_03_29/pipeline.config
ssd_mobilenet_v2_coco_2018_03_29/saved_model/saved_model.pb
ssd_mobilenet_v2_coco_2018_03_29/frozen_inference_graph.pb
ssd_mobilenet_v2_coco_2018_03_29/saved_model/
ssd_mobilenet_v2_coco_2018_03_29/saved_mod

Get its corrensponding .config file.

More can be found from here:
> https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs

In [0]:
!wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config

--2020-03-05 19:10:02--  https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4720 (4.6K) [text/plain]
Saving to: ‘ssd_mobilenet_v2_coco.config’

ssd_mobilenet_v2_co 100%[===================>]   4.61K  --.-KB/s    in 0.001s  

2020-03-05 19:10:03 (8.50 MB/s) - ‘ssd_mobilenet_v2_coco.config’ saved [4720/4720]



# From here we have to do the manual part!

This is a link to my github repository <.....>

Follow the instructions from the 'Process.txt' to pre-process the downloaded road sign dataset. Come back to continue to this notebook only after you have finished pre-processing the dataset

Go to google drive, find the folder that contains everything you have been storing from this notebook. 

> Navigate to models > research > object_detection 

From here create a "dataset" folder and the insert the .record files. 

Also create a "training" folder and insert the "label_map.pbtxt" you would have created by following the instructions

Additionally, download .config model file. Open it with a text editor (such as Sublime) and configure it with the correct checkpoint path, num_classes, num_examples, and label_map_path.

In [0]:
!pwd

/content/drive/My Drive/RoadSignDetection/models/research/object_detection


# After everything has been setup we can now run the following cell where it will begin training and storing the checkpoints to the 'training' folder inside the object_detection

You can leave this running in the background. Aim to reach a loss below 1 and only then stop the execution. This is necessary for sufficient results.

While this is running you can observe the training process by opening the other notebook 'Tensorboard.ipynb' and following the steps

In [0]:
!python legacy/train.py --logtostderr --train_dir=training/ --pipeline_config_path=ssd_mobilenet_v2_coco.config

Streaming output truncated to the last 5000 lines.
I0306 16:29:35.925372 140328148670336 learning.py:507] global step 32881: loss = 1.2021 (0.404 sec/step)
INFO:tensorflow:global step 32882: loss = 1.1199 (0.405 sec/step)
I0306 16:29:36.332181 140328148670336 learning.py:507] global step 32882: loss = 1.1199 (0.405 sec/step)
INFO:tensorflow:global step 32883: loss = 1.0832 (0.403 sec/step)
I0306 16:29:36.736224 140328148670336 learning.py:507] global step 32883: loss = 1.0832 (0.403 sec/step)
INFO:tensorflow:global step 32884: loss = 1.4694 (0.388 sec/step)
I0306 16:29:37.125433 140328148670336 learning.py:507] global step 32884: loss = 1.4694 (0.388 sec/step)
INFO:tensorflow:global step 32885: loss = 1.2858 (0.422 sec/step)
I0306 16:29:37.557415 140328148670336 learning.py:507] global step 32885: loss = 1.2858 (0.422 sec/step)
INFO:tensorflow:global step 32886: loss = 0.9974 (0.531 sec/step)
I0306 16:29:38.122679 140328148670336 learning.py:507] global step 32886: loss = 0.9974 (0.531

In [0]:
!pwd

/content/gdrive/My Drive/Training/models/research/object_detection


# After you have trained the model for a sufficient amount of time and you stopped the execution you are able to export the inference graph which you can use for road sign detection

Go in the training folder, and find the last checkpoint saved. It should be the one with the highest number in model.ckpt-#### (but make sure all 3 files .meta / .index / .data are present) if any of them is missing take the previous checkpoint that has all 3!

Change the following cell commands to that number.

In [0]:
!python export_inference_graph.py \
--input_type image_tensor \
--pipeline_config_path ssd_mobilenet_v2_coco.config \
--trained_checkpoint_prefix training/model.ckpt-35248 \
--output_directory inference_graph_1/

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0306 16:52:16.041875 140021863864192 module_wrapper.py:139] From export_inference_graph.py:145: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0306 16:52:16.049051 140021863864192 module_wrapper.py:139] From /content/drive/My Drive/RoadSignDetection/models/research/object_detection/exporter.py:402: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0306 16:52:16.050594 140021863864192 module_wrapper.py:139] From /content/drive/My Drive/RoadSignDetection/models/research/object_detection/exporter.py:121: The name tf.placeholder is deprecated. Please use tf.comp

# By reaching the end you are now able to run road sign detection on both video and images by using the new frozen_inference_graph that will be stored on the folder you have chosen!